In [1]:
import time; start = time.time()
from datetime import datetime; print(datetime.now().strftime("%B %d, %Y %H:%M:%S"))

June 10, 2024 21:16:22


In [2]:
# CCLE_GeneCorrelation_v01
# Alberto Bejarano 2024

In [3]:
# https://depmap.org/portal/download/all/
# https://s3-eu-west-1.amazonaws.com/pstorage-plus-7003492043/43347204/OmicsExpressionProteinCodingGenesTPMLogp1.csv

In [4]:
%config InlineBackend.figure_format = "retina"
%matplotlib inline

In [5]:
import time; start = time.time()
from datetime import datetime; print(datetime.now().strftime("%B %d, %Y %H:%M:%S"))

June 10, 2024 21:16:22


In [6]:
import pandas as pd
pd.set_option('display.precision', 2)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.precision', 3)
#        -         -        -        -         -        -         -        -        -         -
from scipy.stats import mannwhitneyu, normaltest
from statannotations.Annotator import Annotator
from statannot import add_stat_annotation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas  as pd
import numpy   as np
import mmap
import time
import os
#        -         -        -        -         -        -         -        -        -         -
import warnings; warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [7]:
#Gene_X = 
#Gene_Y =

In [8]:
GenesOfInterest = ['CD274', 'ERBB2', 'MYC', ]   # FERP1

In [9]:
OncotreeCodesOfInterest = ['LUAD','SKCM']

In [10]:
CellLines =  ['A427', 'A549', 'ABC-1', 'COLO 829', 'COLO-679', 'COLO-800', 'COR-L105']

In [11]:
metadata_filev1 = './data/Model_v1.csv'
metadata_filev2 = './data/Model_v2.csv'
data_file       = './data/OmicsExpressionProteinCodingGenesTPMLogp1.csv'

In [12]:
print("File Size:", f"{(os.stat(data_file).st_size/(1024*1024)):,.0f}"," MB")

File Size: 429  MB


In [13]:
meta_dfv1 = pd.read_csv(metadata_filev1, index_col=False, header=0, sep=',', quotechar='"', skiprows=0, on_bad_lines = "warn")
meta_dfv1.sample(3)

,ModelID,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,LegacyMolecularSubtype,PatientMolecularSubtype,RRID,Age,AgeCategory,Sex,PatientRace,PrimaryOrMetastasis,SampleCollectionSite,SourceType,SourceDetail,TreatmentStatus,TreatmentDetails,GrowthPattern,OnboardedMedia,FormulationID,EngineeredModel,TissueOrigin,CCLEName,CatalogNumber,PlateCoating,ModelDerivationMaterial,PublicComments,WTSIMasterCellID,SangerModelID,COSMICID,LegacySubSubtype
134,ACH-000136,PT-3ZPjBh,CHP-126,CHP126,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,MYCN_amp,NaN,CVCL_1123,1.000,Pediatric,Female,NaN,Primary,autonomic_ganglia,DSMZ,,NaN,NaN,Adherent,MF-001-001,RPMI + 10% FBS,NaN,NaN,CHP126_AUTONOMIC_GANGLIA,NaN,NaN,NaN,NaN,38.000,SIDM00286,910567.000,NaN
904,ACH-000908,PT-HKGwye,SNU-520,SNU520,STAD,Esophagus/Stomach,Esophagogastric Adenocarcinoma,Stomach Adenocarcinoma,STAD,NaN,NaN,CVCL_5072,60.000,Adult,Female,asian,Primary,stomach,KCLB,,NaN,NaN,Adherent,MF-001-005,RPMI + 10% FBS + 2mM Glutamine + 25mM HEPES + ...,NaN,NaN,SNU520_STOMACH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1241,ACH-001421,PT-WBS2Hf,WERI-Rb-1,WERIRB1,RBL,Eye,Retinoblastoma,Retinoblastoma,RBL,NaN,NaN,CVCL_1792,1.000,Pediatric,Female,NaN,Primary,eye,ATCC,,NaN,NaN,Suspension,MF-001-001,RPMI + 10% FBS,NaN,NaN,WERIRB1_AUTONOMIC_GANGLIA,HTB-169,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
meta_dfv2 = pd.read_csv(metadata_filev2, index_col=False, header=0, sep=',', quotechar='"', skiprows=0, on_bad_lines = "warn")
meta_dfv2.sample(3)

,ModelID,PatientID,CellLineName,StrippedCellLineName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,LegacyMolecularSubtype,PatientMolecularSubtype,RRID,Age,AgeCategory,Sex,PatientRace,PrimaryOrMetastasis,SampleCollectionSite,SourceType,SourceDetail,TreatmentStatus,TreatmentDetails,GrowthPattern,OnboardedMedia,FormulationID,EngineeredModel,TissueOrigin,CCLEName,CatalogNumber,PlateCoating,ModelDerivationMaterial,PublicComments,WTSIMasterCellID,SangerModelID,COSMICID,LegacySubSubtype
1258,ACH-001481,PT-GWiky5,CHLA-90,CHLA90,NBL,Peripheral Nervous System,Neuroblastoma,Neuroblastoma,NBL,NaN,NaN,CVCL_6610,8.000,Pediatric,Male,NaN,Metastatic,autonomic_ganglia,Academic lab,Bernstein (Icahn School of Medicine at Mount S...,NaN,NaN,Adherent,MF-005-005,IMDM + 20% FBS + 4mM Glutamine + 1x Insulin-Tr...,NaN,NaN,CHLA90_AUTONOMIC_GANGLIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1448,ACH-001857,PT-Sv5Y9u,SG231,SG231,IHCH,Biliary Tract,Intraductal Papillary Neoplasm of the Bile Duct,Intrahepatic Cholangiocarcinoma,IHCH,NaN,NaN,CVCL_0519,42.000,Adult,Male,NaN,Primary,biliary_tract,Academic lab,Bardeesy (MGH),NaN,NaN,Adherent,MF-001-001,RPMI + 10% FBS,NaN,NaN,SG231_BILIARY_TRACT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intrahepatic
1784,ACH-002363,PT-oKf5lW,NCIBL1395,NCIBL1395,ZMNOR,Normal,Non-Cancerous,Matched Normal,NaN,NaN,NaN,CVCL_2638,55.000,Adult,Female,caucasian,NaN,haematopoietic_and_lymphoid_tissue,Academic lab,Sanger,NaN,NaN,Suspension,NaN,NaN,NaN,NaN,NCIBL1395_MATCHED_NORMAL_TISSUE,NaN,NaN,NaN,NaN,NaN,SIDM00643,NaN,NaN


In [15]:
%%time
data_df = pd.read_csv(data_file , index_col=False, header=0, sep=',', quotechar='"', skiprows=0, error_bad_lines=False)

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [16]:
data_df.rename({'Unnamed: 0':'ModelID'},axis=1,inplace=True)

NameError: name 'data_df' is not defined

In [ ]:
data_df.sample(3)

In [ ]:
%%time
transposed_df = data_df.transpose()
transposed_df.reset_index(inplace=True)               # Reset index to make the row names a column
transposed_df.columns = transposed_df.iloc[0]         # Set the first row as column names
transposed_df = transposed_df[1:]                     # Drop the first row which is now the header
transposed_df.reset_index(drop=True, inplace=True)    # Optionally, reset the index again if you want a clean index
transposed_df.rename({'ModelID':'GeneSymbol'},axis=1,inplace=True)
transposed_df['GeneSymbol'] = transposed_df['GeneSymbol'].str.split(' ').str[0]
transposed_df.head(3)

In [ ]:
filtered_df = transposed_df[transposed_df["GeneSymbol"].isin(GenesOfInterest)]

In [ ]:
filtered_df.head()

In [ ]:
exp_data_df = pd.melt(filtered_df, id_vars=['GeneSymbol'],var_name='ModelID',value_name='GeneExpression')
exp_data_df.head()

In [ ]:
meta_df = meta_dfv1[['ModelID', 'CellLineName', 'StrippedCellLineName', 'DepmapModelType', 'OncotreeCode', 'PrimaryOrMetastasis']].copy()

In [ ]:
df = pd.merge(exp_data_df, meta_df, on='ModelID')
order = ['ModelID', 'GeneSymbol', 'CellLineName', 'StrippedCellLineName', 'DepmapModelType', 'OncotreeCode', 'PrimaryOrMetastasis', 'GeneExpression']
df    = df[order]
df.sample()

In [ ]:
%%time
n = 1
for gene in GenesOfInterest:
    data = df[df['GeneSymbol'] == gene]
    sample_counts = data.groupby(['GeneSymbol', 'OncotreeCode']).size().reset_index(name='SampleCount')
    sample_counts = sample_counts.sort_values(by='SampleCount', ascending=False)
    print(sample_counts)
    if n > 0:
        break

In [ ]:
print(df['PrimaryOrMetastasis'].value_counts(dropna=False))

In [ ]:
df.sample(5)

In [ ]:
%%time
df_pivot = df.pivot(index='CellLineName', columns=['GeneSymbol'], values='GeneExpression')
cols = ['CellLineName', 'DepmapModelType', 'OncotreeCode', 'PrimaryOrMetastasis']
#df_merged = pd.merge(df_pivot, df[cols].drop_duplicates(), on='CellLineName')
df_merged = pd.merge(df[cols].drop_duplicates(), df_pivot, on='CellLineName')
df_merged.columns.name = None
df_merged.sample(5)

In [ ]:
filtered_df = df_merged[df_merged["OncotreeCode"].isin(OncotreeCodesOfInterest)]
filtered_df.sample()

In [ ]:
OncotreeCodes = sorted(df_merged['OncotreeCode'].astype(str).unique().tolist());   print('\033[32m', OncotreeCodes, '\033[0m', '\n')
OncotreeCodes1 = sorted(filtered_df['OncotreeCode'].astype(str).unique().tolist()); print('\033[31m', OncotreeCodes1, '\033[0m')

In [ ]:
DepmapModelCodes = sorted(df_merged['DepmapModelType'].astype(str).unique().tolist()); print('\033[32m', DepmapModelCodes, '\033[0m', '\n')
DepmapModelCodes = sorted(filtered_df['DepmapModelType'].astype(str).unique().tolist()); print('\033[31m',DepmapModelCodes, '\033[0m')

In [ ]:
CellLinesOfInterest  = sorted(filtered_df['CellLineName'].astype(str).unique().tolist()); print('\033[32m', CellLinesOfInterest, '\033[0m', '\n')
CellLinesOfInterest = CellLines ; print('\033[31m', CellLinesOfInterest, '\033[0m')

In [ ]:
df_merged['PrimaryOrMetastasis'] = df_merged['PrimaryOrMetastasis'].fillna('Unknown')
Primary_Metastasis = sorted(df_merged['PrimaryOrMetastasis'].astype(str).unique().tolist()); print('\033[31m',Primary_Metastasis, '\033[0m')

In [ ]:
unique_values = df_merged['PrimaryOrMetastasis'].unique(); print(unique_values)
palette = sns.color_palette("hsv", len(unique_values))

In [ ]:
genes = sorted(df['GeneSymbol'].astype(str).unique().tolist()); print('\033[33m', genes, '\033[0m')

In [ ]:
hue_order = ["Primary", "Metastasis"]

In [ ]:
palette = {
    'Primary':    '#1f77b4',  # blue
    'Metastatic': '#ff7f0e',  # orange
    'Unknown':    '#2ca02c',  # green (for example, if needed)
}

In [ ]:
hex_color_01 = "#7E7E7E"  # For example, you can choose any hex color
hex_color_02 = "#FF0000"
#          -          -          -          -          -          -
palette   = {code:hex_color_01 for code in OncotreeCodes}
#          -          -          -          -          -          -
for key in OncotreeCodesOfInterest:
    print(key)
    palette[key] = hex_color_02

In [ ]:
print

In [ ]:
data = df_merged
scatter_kws = {'s':6, 'alpha':0.25, 'linewidth':0.2, 'edgecolor':'#1E1E1E', 'palette':palette}
kws1 = {'s':25, 'alpha':0.25, 'linewidth':0.5,'edgecolor':'#1E1E1E', 'palette':palette}
line_kas= {'color': 'red', 'alpha':0.25, 'lw':1};

In [ ]:
for i in range(len(genes)):
    for j in range(i + 1, len(genes)):
        #          -          -          -          -          -          -          -          -          -          -          -          -
        figsize=(12/2.54, 12/2.54)
        a = genes[i]; b = genes[j]; print(a, b)
        #          -          -          -          -          -          -          -          -          -          -          -          -
        fig, ax = plt.subplots(figsize=figsize, dpi=300)
        plt.subplots_adjust(wspace=.1, hspace=.1); #plt.axis('off')
        ax = sns.scatterplot(data=data, x=data[a], y=data[b], hue='OncotreeCode', **scatter_kws)
        #                    -          -          -          -          -          -          -          -          -          -          -          -          -          -  
        ax.set_title(f"{a} vs. {b}", fontsize=10, color="#7E7E7E", pad=8) 
        axlabelkws = {'fontsize':8, 'color':'#7E7E7E', 'alpha':0.8, 'labelpad':4}
        plt.xlabel(a, **axlabelkws); plt.ylabel(b, **axlabelkws)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)
        for spine in ['left', 'bottom']:
            ax.spines[spine].set_color('#7E7E7E'); ax.spines[spine].set_alpha(0.3);
            ax.spines[spine].set_linewidth(0.5)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
        data2 = data.copy()
        types = data2.reset_index()['CellLineName'].values; types=types[:5]; #print(types)
        x = data2[a].values; y = data2[b].values; types = CellLinesOfInterest
        for e, txt in enumerate(types):
            #print(e); print(txt)
            ax.annotate(txt, (x[e], y[e]), xytext=figsize, textcoords='offset pixels', fontsize=3, weight=500, color="#000000",)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
        plt.xlim(-1,14); plt.ylim(-1,14);
        ax.tick_params(axis="x", colors="#cfcfcf", pad=2, length=1);      ax.tick_params(axis="y", which='major', colors="#cfcfcf", pad=5, length=1)
        plt.xticks(fontsize=6, color="#7E7E7E", rotation=0, ha="center"); plt.yticks(fontsize=6, color="#7E7E7E", rotation=0, ha="center"); 
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
        kws_lines = {"ls":"--", "alpha":0.5, "linewidth":0.1}
        plt.axvline(x=0.584, color="g", **kws_lines); plt.axvline(x=1.32, color="orange", **kws_lines); plt.axvline(x=10, color="red", **kws_lines)
        plt.axhline(y=0.584, color="g", **kws_lines); plt.axhline(y=1.32, color="orange", **kws_lines); plt.axhline(y=10, color="red", **kws_lines)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          - 
        handles, labels = ax.get_legend_handles_labels()
        plt.legend(handles[0:2],labels[0:2], frameon=False, fontsize=10, bbox_to_anchor=(0.1,0.1), loc=0, borderaxespad=0.1, facecolor="white", edgecolor="#f6f6f6",)
        plt.legend([],[])
        plt.tight_layout()
        plt.show(); plt.close()

In [ ]:
data = filtered_df

In [ ]:
for i in range(len(genes)):
    for j in range(i + 1, len(genes)):
        #          -          -          -          -          -          -          -          -          -          -          -          -
        figsize=(12/2.54, 12/2.54)
        a = genes[i]; b = genes[j]; print(a, b)
        #          -          -          -          -          -          -          -          -          -          -          -          -
        fig, ax = plt.subplots(figsize=figsize, dpi=300)
        plt.subplots_adjust(wspace=.1, hspace=.1); plt.axis('off')
        ax = sns.scatterplot(data=data, x=data[a], y=data[b], hue='OncotreeCode', **scatter_kws)

        #                    -          -          -          -          -          -          -          -          -          -          -          -          -          -  
        ax.set_title(f"{a} vs. {b}", fontsize=10, color="#7E7E7E", pad=8) 
        axlabelkws = {'fontsize':8, 'color':'#7E7E7E', 'alpha':0.8, 'labelpad':6}
        plt.xlabel("index", **axlabelkws); plt.ylabel("Correlation", **axlabelkws)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)
        for spine in ['left', 'bottom']:
            ax.spines[spine].set_color('#7E7E7E'); ax.spines[spine].set_alpha(0.3);
            ax.spines[spine].set_linewidth(0.5)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
        data2 = data.copy()
        types = data2.reset_index()['CellLineName'].values; types=types[:5]; #print(types)
        x = data2[a].values; y = data2[b].values; types = CellLinesOfInterest
        for e, txt in enumerate(types):
            #print(e); print(txt)
            ax.annotate(txt, (x[e], y[e]), xytext=figsize, textcoords='offset pixels', fontsize=4, weight=400, color="#7E7E7E",)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
        ax.tick_params(axis="x", colors="#cfcfcf", pad=2, length=1);      ax.tick_params(axis="y", which='major', colors="#cfcfcf", pad=5, length=3)
        plt.xticks(fontsize=6, color="#7E7E7E", rotation=0, ha="center"); plt.yticks(fontsize=6, color="#7E7E7E", rotation=0, ha="center"); 
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
        handles, labels = ax.get_legend_handles_labels()
        plt.legend(handles[0:2],labels[0:2], frameon=False, fontsize=10, bbox_to_anchor=(0.1,0.1), loc=0, borderaxespad=0.1)
        plt.legend(fontsize=8, bbox_to_anchor=(0.95,0.95), loc=0, borderaxespad=0.1, facecolor="white", edgecolor="#f6f6f6", frameon=True)
        plt.legend([],[])
        plt.tight_layout()
        plt.show(); plt.close()

In [ ]:
print("'BQ_ISB.CGC_TCGA_GeneExprCorrelation_v01' script run time:", f'{time.time()-start:.0f}', "seconds.")

In [ ]:
iuhiuh

In [ ]:
for i in range(len(genes)):
    for j in range(i + 1, len(genes)):
        #          -          -          -          -          -          -          -          -          -          -          -          -
        figsize=(12/2.54, 12/2.54)
        a = genes[i]; b = genes[j]; #print(a, b); 
        fig, ax = plt.subplots(figsize=figsize, dpi=300)
        plt.subplots_adjust(wspace=.1, hspace=.1); plt.axis('off')
        ax = sns.scatterplot(data=data, x=data[a], y=data[b], hue='PrimaryOrMetastasis', **scatter_kws)

        #                    -          -          -          -          -          -          -          -          -          -          -          -          -          -  
        ax.set_title(f"{a} vs. {b}", fontsize=10, color="#7E7E7E", pad=8) 
        axlabelkws = {'fontsize':8, 'color':'#7E7E7E', 'alpha':0.8, 'labelpad':6}
        plt.xlabel("index", **axlabelkws); plt.ylabel("Correlation", **axlabelkws)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)
        for spine in ['left', 'bottom']:
            ax.spines[spine].set_color('#7E7E7E'); ax.spines[spine].set_alpha(0.3);
            ax.spines[spine].set_linewidth(0.5)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
        data2 = data.copy()
        types = data2.reset_index()['CellLineName'].values; types=types[:5]; #print(types)
        x = data2[a].values; y = data2[b].values
        for i, txt in enumerate(types):
            #print(i); print(txt)
            ax.annotate(txt, (x[i], y[i]), xytext=figsize, textcoords='offset pixels', fontsize=6, weight=400, color="#7E7E7E",)
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -        
        ax.tick_params(axis="x", colors="#cfcfcf", pad=2, length=1);      ax.tick_params(axis="y", which='major', colors="#cfcfcf", pad=5, length=3)
        plt.xticks(fontsize=6, color="#7E7E7E", rotation=0, ha="center"); plt.yticks(fontsize=6, color="#7E7E7E", rotation=0, ha="center"); 
        #          -          -          -          -          -          -          -          -          -          -          -          -          -          -   
        handles, labels = ax.get_legend_handles_labels()
        plt.legend(handles[0:2],labels[0:2], frameon=False, fontsize=10, bbox_to_anchor=(0.1,0.1), loc=0, borderaxespad=0.1)
        plt.legend(fontsize=8, bbox_to_anchor=(0.95,0.95), loc=0, borderaxespad=0.1, facecolor="white", edgecolor="#f6f6f6", frameon=True)

        plt.tight_layout()
        plt.show(); plt.close()

In [ ]:
%%time
for gene in GenesOfInterest:
    fig, ax = plt.subplots(figsize=(12, 6)); plt.subplots_adjust(wspace=0.02, hspace=0.01)
    sns.plotting_context("notebook", font_scale = 2)
    #        -         -        -        -         -        -         -        -        -         -        -        -         -    
    data = df[df['GeneSymbol'] == gene]
    sns.stripplot(data=data, x='OncotreeCode', y='GeneExpression')

In [ ]:
%%time
for gene in GenesOfInterest:
    fig, ax = plt.subplots(figsize=(12, 6)); plt.subplots_adjust(wspace=0.02, hspace=0.01)
    sns.plotting_context("notebook", font_scale = 2)
    #        -         -        -        -         -        -         -        -        -         -        -        -         -    
    data = df[df['GeneSymbol'] == gene]

    median_df = data.groupby(['OncotreeCode'])['GeneExpression'].median().sort_values(ascending=False).reset_index()
    order = median_df.OncotreeCode.tolist()


    
    data=data; x="OncotreeCode"; y = "GeneExpression"; hue='PrimaryOrMetastasis'
    
    str_kws = {'s':6, 'jitter':.25, 'alpha':0.5, 'dodge':True, 'linewidth':0.2,'edgecolor':'#1E1E1E','palette':palette,
               'order':order, 'hue':hue}
    plt.gca().set_yscale('symlog'); plt.ylim(-0.2, 1E8) 
    g = sns.stripplot(data=data, x=x, y=y, zorder=0, **str_kws, ax=ax)

In [ ]:
%%time
for gene in GenesOfInterest:
    fig, ax = plt.subplots(figsize=(12, 6)); plt.subplots_adjust(wspace=0.02, hspace=0.01)
    sns.plotting_context("notebook", font_scale = 2)
    #        -         -        -        -         -        -         -        -        -         -        -        -         -    
    data = df[df['GeneSymbol'] == gene]
    #data.dropna(subset=['GeneExpression'], inplace=True)
    #        -         -        -        -         -        -         -        -        -         -        -        -         -
    #        -         -        -        -         -        -         -        -        -         -        -        -         -
    median_df = data.groupby(['OncotreeCode'])['GeneExpression'].median().sort_values(ascending=False).reset_index()
    order = median_df.OncotreeCode.tolist()
    #        -         -        -        -         -        -         -        -        -         -        -        -         -
    data=data; x="OncotreeCode"; y = "GeneExpression"; hue='PrimaryOrMetastasis'; hue_order=hue_order
    #        -         -        -        -         -        -         -        -        -         -        -        -         -   
    #annot_kws1 = {'pairs':pairs, 'data':data, 'x':x, 'y':y, 'order':order, 'hue':hue, 'hue_order':hue_order}
    #annot_params = {"test":"Mann-Whitney", "line_width":1, "color":"#7E7E7E", "text_format":"star", 'loc':'inside'}
    #        -         -        -        -         -        -         -        -        -         -        -        -         -   
    str_kws = {'s':6, 'jitter':.25, 'alpha':0.5, 'dodge':True, 'linewidth':0.2,'edgecolor':'#1E1E1E','palette':palette,
               'order':order, 'hue':hue, 'hue_order':hue_order}
    box_kws = {'linewidth':0.5, 'width':0.75, 'dodge':True, 'showcaps':True, 'whis':1.5, 'fliersize':0, 'saturation':0.20, 
               'order':order, 'hue_order':hue_order}
    mprops=dict(linestyle='-', linewidth=1, color="firebrick", alpha=0.5); cprops=dict(linestyle="-", alpha=.5, linewidth=.3);
    bprops=dict(linestyle="-", alpha=.3, linewidth=.3); wprops=dict(linestyle="--", linewidth=.3, color="#7E7E7E");
    #        -         -        -        -         -        -         -        -        -         -        -        -         -   
    plt.gca().set_yscale('symlog'); plt.ylim(-0.2, 1E8) 
    g = sns.stripplot(data=data, x=x, y=y, zorder=0, **str_kws, ax=ax)   
    g = sns.boxplot(data=data,x=x, y=y, hue=hue, medianprops=mprops, capprops=cprops, boxprops=bprops, whiskerprops=wprops, **box_kws, zorder=1, ax=ax)    
    #        -         -        -        -         -        -         -        -        -         -        -        -         -   
    #annot_kws1 = {'pairs':pairs, 'data':data, 'x':x, 'y':y, 'order':order, 'hue':hue, 'hue_order':hue_order}
    #annot_params = {"test":"Mann-Whitney", "line_width":1, "color":"#7E7E7E", "text_format":"star", 'loc':'inside'}
    #        -         -        -        -         -        -         -        -        -         -        -        -         -   
    str_kws = {'s':6, 'jitter':.25, 'alpha':0.5, 'dodge':True, 'linewidth':0.2,'edgecolor':'#1E1E1E','palette':palette,
               'order':order, 'hue':hue, 'hue_order':hue_order}
    box_kws = {'linewidth':0.5, 'width':0.75, 'dodge':True, 'showcaps':True, 'whis':1.5, 'fliersize':0, 'saturation':0.20, 
               'order':order, 'hue_order':hue_order}
    mprops=dict(linestyle='-', linewidth=1, color="firebrick", alpha=0.5); cprops=dict(linestyle="-", alpha=.5, linewidth=.3);
    bprops=dict(linestyle="-", alpha=.3, linewidth=.3); wprops=dict(linestyle="--", linewidth=.3, color="#7E7E7E");
    #        -         -        -        -         -        -         -        -        -         -        -        -         -   